In [53]:
import csv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import pandas as pd
from io import StringIO


In [54]:
# Set up your Spotify API credentials
client_id = "83578afe5b1d4137a5e813cac12a370a"
client_secret = "3c4427427adb4a269d0c207b51f3c68d"

# Set the input and output CSV file names
input_file = "billboard_hot_100_2020_2023.csv"
output_file = "billboard_hot_100_audio_features.csv"

# Set up the Spotify API client
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Read the unique songs from the input CSV file as a DataFrame
songs_df = pd.read_csv(input_file)


In [58]:
# Change "Featuring" to "feat." in billboard dataset for all artists
songs_df['Artist'] = songs_df['Artist'].replace('Featuring', 'feat.', regex=True)

# Remove brackets and & from dataframe
songs_df['Artist'] = songs_df['Artist'].str.replace(r"\(.*\)","")
songs_df['Artist'] = songs_df['Artist'].str.replace(r"\&.*","")



songs_df.head()


/var/folders/xt/0h_f_zjs1hvcdg6rdl1hs5_c0000gn/T/ipykernel_42239/3980433899.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  songs_df['Artist'] = songs_df['Artist'].str.replace(r"\(.*\)","")
/var/folders/xt/0h_f_zjs1hvcdg6rdl1hs5_c0000gn/T/ipykernel_42239/3980433899.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  songs_df['Artist'] = songs_df['Artist'].str.replace(r"\&.*","")


,Title,Artist
0,Flossin',Internet Money feat. YoungBoy Never Broke Again
1,Honest,Justin Bieber
2,Premonition (Intro),Eminem
3,Franchise,Travis Scott feat. Young Thug
4,Getting Older,Billie Eilish


In [59]:
# Fetch audio features for the songs and store them in a list
song_audio_features = []
for index, song in songs_df.iterrows():
    title = song["Title"]
    artist = song["Artist"]

    # Search for the song on Spotify
    query = f"track:{title} artist:{artist}"
    search_results = sp.search(query, type="track", limit=1)

    if search_results["tracks"]["items"]:
        track = search_results["tracks"]["items"][0]
        track_id = track["id"]

        # Get the audio features for the song
        audio_features = sp.audio_features(track_id)[0]

        # Save the song data and audio features
        song_data = {
            "title": title,
            "artist": artist,
            "spotify_id": track_id,
            "audio_features": audio_features
        }

        song_audio_features.append(song_data)
        print(f"Added audio features for {title} by {artist}")
    else:
        print(f"Song not found on Spotify: {title} by {artist}")

# Create a DataFrame with the song data and audio features
audio_features_columns = list(song_audio_features[0]["audio_features"].keys())
output_columns = ["title", "artist", "spotify_id"] + audio_features_columns

output_data = [
    {
        "title": song_data["title"],
        "artist": song_data["artist"],
        "spotify_id": song_data["spotify_id"],
        **song_data["audio_features"]
    }
    for song_data in song_audio_features
]

output_df = pd.DataFrame(output_data, columns=output_columns)

# Write the DataFrame to a CSV file
output_df.to_csv(output_file, index=False)

print(f"Exported audio features to {output_file}")


Added audio features for Flossin' by Internet Money feat. YoungBoy Never Broke Again
Added audio features for Honest by Justin Bieber 
Added audio features for Premonition (Intro) by Eminem
Added audio features for Franchise by Travis Scott feat. Young Thug 
Added audio features for Getting Older by Billie Eilish
Added audio features for Like It's Christmas by Jonas Brothers
Added audio features for Woods by Mac Miller
Added audio features for Without You by The Kid LAROI
Added audio features for Mr. Jones by Pop Smoke feat. Future
Added audio features for Lotus by Lil Uzi Vert
Song not found on Spotify: Let Somebody Go by Coldplay X Selena Gomez
Added audio features for Do It On The Tip by Megan Thee Stallion feat. City Girls 
Added audio features for Moth To A Flame by Swedish House Mafia 
Added audio features for Humble by Lil Baby
Added audio features for Livin' The Dream by Morgan Wallen
Added audio features for KEII by Anuel AA
Song not found on Spotify: I Like You (A Happier Son